### Imports

In [349]:
import numpy as np
from PIL import Image, ImageDraw

### Galois Field Math

In [350]:
def GF_add(a, b, F):
    return (a + b) % F

def GF_sub(a, b, F):
    return (a - b) % F

def GF_mul(a, b, F):
    return (a * b) % F

def GF_div(a, b, F):
    return (a * b ** (F - 2)) % F

def GF_pow(a, b, F):
    return (a ** b) % F

def GF_eval(p, x, F):
    y = 0
    for i in range(len(p)):
        y = GF_add(y, GF_mul(p[i], GF_pow(x, i, F), F), F)
    return y

### Encoding a message vector using Reed-Solomon codes

#### Code parameters

In [351]:
F = 11 # Galois Field F11
n = 11 # Size of codeword (the encoded message)
k = 7 # Size of message (the original message)
t = 2 # Error correction capability

eval_points = np.arange(0, F) # Evaluation points
print("Evaluation points: ", eval_points)

Evaluation points:  [ 0  1  2  3  4  5  6  7  8  9 10]


In [385]:
msg1 = np.array([2, 10, 3, 7, 5, 1, 9])
print("Message 1: ", msg1)

msg2 = np.array([7, 2, 9, 4, 3, 0, 1])
print("Message 2: ", msg2)

msg3 = np.array([3, 5, 8, 2, 5, 10, 9])
print("Message 3: ", msg3)

Message 1:  [ 2 10  3  7  5  1  9]
Message 2:  [7 2 9 4 3 0 1]
Message 3:  [ 3  5  8  2  5 10  9]


In [363]:
def encode(msg, eval_points):
    encoded_msg = np.zeros(n)
    for i in range(n):
        x = eval_points[i]
        for j in range(k):
            encoded_msg[i] += msg[j] * x**j
        encoded_msg %= F
        # encoded_msg[i] = (msg[0] + msg[1] * x + msg[2] * x**2 + msg[3] * pow(x, 3) + msg[4] * pow(x, 4)) % F

    return encoded_msg.astype(int)

In [354]:
def print_encoded_msg_binary(encoded_msg):
    bin_msg = []
    bin_msg.append("010")
    for i in range(n):
        bin_msg.append('{0:04b}'.format(int(encoded_msg[i])))
    bin_msg = np.array(bin_msg)
    print(bin_msg)

    binary_string = "".join(bin_msg)
    print(binary_string)

    return binary_string

In [386]:
encoded_msg1 = encode(msg1, eval_points)
print("Encoded Message 1: ", encoded_msg1)
binary_string1 = print_encoded_msg_binary(encoded_msg1)

encoded_msg2 = encode(msg2, eval_points)
print("\nEncoded Message 2: ", encoded_msg2)
binary_string2 = print_encoded_msg_binary(encoded_msg2)

encoded_msg3 = encode(msg3, eval_points)
print("\nEncoded Message 3: ", encoded_msg3)
binary_string3 = print_encoded_msg_binary(encoded_msg3)

Encoded Message 1:  [ 2  4  8 10  7  4  0  8 10  1  1]
['010' '0010' '0100' '1000' '1010' '0111' '0100' '0000' '1000' '1010'
 '0001' '0001']
01000100100100010100111010000001000101000010001

Encoded Message 2:  [ 7  4  4  8 10  4  7 10  0  9  3]
['010' '0111' '0100' '0100' '1000' '1010' '0100' '0111' '1010' '0000'
 '1001' '0011']
01001110100010010001010010001111010000010010011

Encoded Message 3:  [ 3  9  3  3 10  6  8  2 10  4  8]
['010' '0011' '1001' '0011' '0011' '1010' '0110' '1000' '0010' '1010'
 '0100' '1000']
01000111001001100111010011010000010101001001000


In [356]:
width, height = len(binary_string1), 100
img = Image.new("RGB", (width, height), "white")

draw = ImageDraw.Draw(img)
for i, b in enumerate(binary_string1):
    x0, y0 = i, 0
    x1, y1 = i + 1, height
    if b == "1":
        draw.rectangle([x0, y0, x1, y1], fill="black")
    else:
        draw.rectangle([x0, y0, x1, y1], fill="white")

img.save("bar_code1.png")

width, height = len(binary_string2), 100
img = Image.new("RGB", (width, height), "white")

draw = ImageDraw.Draw(img)
for i, b in enumerate(binary_string2):
    x0, y0 = i, 0
    x1, y1 = i + 1, height
    if b == "1":
        draw.rectangle([x0, y0, x1, y1], fill="black")
    else:
        draw.rectangle([x0, y0, x1, y1], fill="white")

img.save("bar_code2.png")

width, height = len(binary_string3), 100
img = Image.new("RGB", (width, height), "white")

draw = ImageDraw.Draw(img)
for i, b in enumerate(binary_string3):
    x0, y0 = i, 0
    x1, y1 = i + 1, height
    if b == "1":
        draw.rectangle([x0, y0, x1, y1], fill="black")
    else:
        draw.rectangle([x0, y0, x1, y1], fill="white")

img.save("bar_code3.png")

### Introducing errors

In [404]:
received_vector = encoded_msg1.copy()
# received_vector[1] = 1
# received_vector[2] = 2
pos = 1
received_vector[5] = GF_add(received_vector[5], 8, F)
# received_vector[10] = GF_add(received_vector[10], 2, F)
print(encoded_msg1)
print(received_vector)

[ 2  4  8 10  7  4  0  8 10  1  1]
[ 2  4  8 10  7  1  0  8 10  1  1]


In [412]:
binary_rec_vector = ['0010', '0100', '1000', '1010', '0111', '0101', '0000', '1000', '1010', '0001', '0001']

received_vector = np.array([int(binary_rec_vector[i], 2) for i in range(len(binary_rec_vector))])
received_vector

array([ 2,  4,  8, 10,  7,  5,  0,  8, 10,  1,  1])

#### Syndrome calculation
* To find syndrome, we first find $H$ (the parity check matrix) and then multiply it with the received vector $r$.

In [405]:
# generator matrix
G =[]
for point in eval_points:
    G.append(np.array([GF_pow(point, i, F) for i in range(k)]))
G = np.array(G)
G = G.T
print("G: ", G)

G:  [[ 1  1  1  1  1  1  1  1  1  1  1]
 [ 0  1  2  3  4  5  6  7  8  9 10]
 [ 0  1  4  9  5  3  3  5  9  4  1]
 [ 0  1  8  5  9  4  7  2  6  3 10]
 [ 0  1  5  4  3  9  9  3  4  5  1]
 [ 0  1 10  1  1  1 10 10 10  1 10]
 [ 0  1  9  3  4  5  5  4  3  9  1]]


In [368]:
# parity check matrix
H = []
for point in eval_points:   
    H.append(np.array([GF_pow(point, i, F) for i in range(n-k)]))
H = np.array(H)
H = H.T
print("H: ", H)

H:  [[ 1  1  1  1  1  1  1  1  1  1  1]
 [ 0  1  2  3  4  5  6  7  8  9 10]
 [ 0  1  4  9  5  3  3  5  9  4  1]
 [ 0  1  8  5  9  4  7  2  6  3 10]]


In [369]:
# veryfying if G * H.T = 0
np.dot(G, H.T) % F

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [413]:
# syndrome calculation
syndromes = np.dot(received_vector, H.T) % F
print("Syndromes: ", syndromes)

Syndromes:  [1 5 3 4]


#### Berlekamp-Massey algorithm to find error locator polynomial

In [371]:
def find_error_locator(syndromes, eval_points):
    l = 0
    r = 0
    L = np.array([1])
    R = [1]

    while r < 2*t:
        r += 1
        desc = 0
        for i in range(l + 1):
            if len(L) < l+1:
                L = np.append(L, np.zeros(l+1 - len(L)))
            desc = GF_add(desc, GF_mul(L[i], syndromes[r-i-1], F), F)

        if desc == 0:
            continue
        else:
            if isinstance(R, np.ndarray):
                R = R.tolist()
            L_new = [0] + R
            L_new = desc * np.array(L_new)

            if len(L_new) > len(L):
                L = np.append(L, np.zeros(len(L_new) - len(L)))
            else:
                L_new = np.append(L_new, np.zeros(len(L) - len(L_new)))

            L_old = L
            L = GF_sub(L, L_new, F).astype(int)

            if 2*l <= r-1:
                l = r - l
                R = GF_div(L_old, desc, F).astype(int)
            else:
                R = [0] + R

    return L

In [414]:
error_locator = find_error_locator(syndromes, eval_points)
print("Error locator: ", error_locator)

Error locator:  [1 6 0]


#### Chien search to find error positions
* This finds the roots of the error locator polynomial, which are the error positions.

In [373]:
def find_error_positions(error_locator, eval_points):
    error_positions = []
    for i in range(len(eval_points)):
        if GF_eval(error_locator, eval_points[i], F) == 0:
            error_positions.append(i)
    error_positions = np.array(error_positions)
    return error_positions

In [415]:
error_positions = find_error_positions(error_locator, eval_points)
print("Error positions: ", error_positions)

Error positions:  [9]


#### Error evaluator polynomial

In [375]:
def find_error_evaluator(error_locator, syndromes):
    L = error_locator

    error_evaluator = np.zeros(len(L) + len(syndromes) - 1)
    for i in range(len(L)):
        for j in range(len(syndromes)):
            error_evaluator[i+j] = GF_add(error_evaluator[i+j], GF_mul(L[i], syndromes[j], F), F)

    error_evaluator = error_evaluator[:2*t].astype(int)
    return error_evaluator

In [416]:
error_evaluator = find_error_evaluator(error_locator, syndromes)
print("Error evaluator: ", error_evaluator)

Error evaluator:  [1 0 0 0]


#### Forney algorithm to find error values
* This finds the error values at the error positions.
* Error positions are: $e_{k_j} = \frac{-1}{x_j}\frac{\Omega(x_j)}{\Lambda'(x_j)}$, where $x_j$ are the error positions, $\Omega(x)$ is the error evaluator polynomial, and $\Lambda'(x)$ is the derivative of the error locator polynomial.

In [377]:
def find_error_values(error_locator, error_evaluator, error_positions):
    numerator = GF_eval(error_evaluator, error_positions, F).astype(int)
    L_dash = error_locator[1:] # derivative of error locator polynomial
    denominator = GF_eval(L_dash, error_positions, F).astype(int)

    error_values = GF_add(F - GF_div(GF_div(numerator, denominator, F), error_positions, F), 0, F)

    return error_values

In [417]:
error_values = find_error_values(error_locator, error_evaluator, error_positions)
print("Error values: ", error_values)

Error values:  [1]


In [418]:
corrected_vector = received_vector.copy()
corrected_vector[error_positions] = GF_sub(corrected_vector[error_positions], error_values, F)
print("Corrected vector: ", corrected_vector)

Corrected vector:  [ 2  4  8 10  7  5  0  8 10  0  1]


In [383]:
def decode(corrected_vector):
    # Get the message from the corrected vector
    # it is not the first k elements of the corrected vector
    # encoding is done by evaluating the polynomial at 'n' evaluation points

    decoded_msg = []
    for i in range(n):
        decoded_msg.append(GF_eval(corrected_vector, eval_points[i], F))
    decoded_msg = np.array(decoded_msg[:k])

    return decoded_msg

In [384]:
print("Original message: ", msg1)
decoded_msg = decode(corrected_vector)
print("Decoded message: ", decoded_msg)

Original message:  [ 2 10  3  7  5  1  9]
Decoded message:  [ 2  0  4  9 10  7  0]
